<a href="https://colab.research.google.com/github/ramprakashyadav/OCR/blob/main/redbull_model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber

In [ ]:
import pdfplumber
import os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/My Drive/redbull_model2/

/content/gdrive/My Drive/redbull_model2


In [ ]:
INPUT_FOLDER = "/content/gdrive/MyDrive/redbull_model2/input_folder/"
OUTPUT_FOLDER = "/content/gdrive/MyDrive/redbull_model2/output_text/"

In [ ]:
for fl in os.listdir(INPUT_FOLDER):

  pdf = pdfplumber.open(INPUT_FOLDER+fl)
  pdf_text=pdf.pages[0].extract_text()
  with open(OUTPUT_FOLDER + fl.replace('.pdf','.txt'),'w') as f:
    f.write(pdf_text)

In [ ]:
!pip install -U spacy -q
!python -m spacy info

     |████████████████████████████████| 6.0 MB 23.6 MB/s 
     |████████████████████████████████| 628 kB 48.1 MB/s 
     |████████████████████████████████| 451 kB 51.1 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 181 kB 59.8 MB/s 
     |████████████████████████████████| 10.1 MB 53.3 MB/s 

============================== Info about spaCy ==============================

spaCy version    3.2.2                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.12                        
Pipelines                                      



In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en")
db = DocBin()

In [ ]:
import json
f = open('training_data.json')
TRAIN_DATA = json.load(f)

In [ ]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
          print("Skipping entity")
        else:
          ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy")

100%|██████████| 50/50 [00:00<00:00, 1571.04it/s]


In [ ]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-02-12 07:51:38,921] [INFO] Set up nlp object from config
[2022-02-12 07:51:38,934] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-02-12 07:51:38,939] [INFO] Created vocabulary
[2022-02-12 07:51:38,940] [INFO] Finished initializing nlp object
[2022-02-12 07:51:39,161] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     52.19    0.00    0.00    0.00    0.00
 47     200         19.18    876.59   90.91  100.00   83.33    0.91
104     400        182.69    129.03   90.91  100.00   83.33    0.91
171     600        108.12    128.78   90.91  100

In [ ]:
nlp_ner = spacy.load("/content/gdrive/MyDrive/redbull_model2/model-best")

In [ ]:
doc =nlp_ner('''RedBull India Pvt. Ltd
c/o AWL India Pvt Ltd, Khasra no. 3401 JA,
Village Amausi, Industrial Area,, Nadarganj, District, Uttar
Pradesh
Lucknow -226008 | State Code: 9 | GSTIN:
09AADCR5842H1Z6
From DELIVERY CHALLAN
RedBull India Pvt. Ltd
c/o AWL India Pvt Ltd, Khasra no. 3401 JA, Document No.: 4203401401
Village Amausi, Industrial Area,, Nadarganj, District, Uttar Document Date: 04-09-2021
Pradesh Ref. No.
Lucknow : 226008 Ref Date:
From State Code: 9 LR no: LKO010004721
GSTIN:09AADCR5842H1Z6 Vehicle No: UP32KN3066
To Transporter: 09AAGCA4705P1ZB
Mode of Transport:Road
RedBull India Pvt Ltd TransporterName: AWL INDIA PVT LTD
C-58 Ground Floor, Durga Tower
Vibhuti Khand, Uttar Pradesh
Gomtinagar Lucknow - 226010
Serial No Material Description HSN/SAC Qty Rate Per Amount
Red Bull Energy Drink IN 250ml
RB30925
10 22029990 50 734.92 BOX 36,746.16
Mfg. Dt.: 01.03.2021-50
Red Bull IN Sugar Free 250ml WA
RB205906
20 22029990 20 734.92 BOX 14,698.46
Mfg. Dt.: 30.04.2021-20
Red Bull ED IN Alu Can 24x330ml WA
RB217737
30 22029990 40 970.10 BOX 38,803.95
Mfg. Dt.: 06.12.2019-40
RB Yellow Edition IN Alu Can 24x250ml
RB231471
40 22029990 20 734.92 BOX 14,698.46
Mfg. Dt.: 23.02.2021-20
Total Value 104,947.03
Amount in words:INR One Lakh Four Thousand Nine Hundred Forty Seven and Three Paisa Only
No. of Box: 130
Gross weight - 962.400
FSSAI Lic. No - 12721066000743
For Red Bull India Pvt. Ltd.
Authorised signatory
Declaration:
We declare that this document is for supply of goods between distinct or related person other than through an agent and the receipient is eligible for full input tax credit.I / We hereby
certify that food/foods mentioned in this document is / are warranted to be of the nature and quality which it/these purports/purport to be.
Red Bull India Pvt. Ltd.
Registered Office Address: 1st Floor, B Wing, 215 Atrium, Kanakia Spaces, Chakala, Andheri Kurla Road, Andheri (E) Mumbai-400093. www.redbull.in
Tel:+91-22-66180888, Fax:+91-22-66180808, Email: info.in@redbull.com, CIN: U51909MH2007FTC169580 PAN: AADCR5842H''')

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True)

In [ ]:
import pandas as pd

In [ ]:
all_data =[]
for fl in os.listdir(INPUT_FOLDER):
    print(fl)
    pdf_obj = pdfplumber.open(INPUT_FOLDER + fl)
    for page in pdf_obj.pages:
        tables = page.extract_tables()
        page_txt = page.extract_text()
        # print(page_txt)
        for table in tables:
            if 'DELIVERY' in ''.join(table[0]):
                document_no =  table[1][0].split('\n')[0].replace('Document No.:','').strip()
                print(document_no)
            if 'Serial No' in ''.join(table[0]) or 'Item' in ''.join(table[0]):
                table_df = pd.DataFrame(table[1::],columns=table[0])
                # print(table_df.columns)
                for idx, row in table_df.iterrows():
                    if row['Serial No'].isnumeric():
                        # print(type(row['Serial No']))
#                         print(idx)
                        if idx == 0:
                            doc = nlp_ner(page_txt)
                            all_label_text = []
                            for ent in doc.ents:
                              print(ent.label_)
                              if ent.label_ == "COMPANY NAME":
                                company_name = ent.text
                              if ent.label_ == "COMPANY NAME":
                                company_name = ent.text
                              if ent.label_ == "DOCUMENT DATE":
                                document_date = ent.text
                              if ent.label_ == "COMPANY GSTIN NUMBER":
                                company_gstin = ent.text
                              if ent.label_ == "VEHICLE NO":
                                vehicle_no = ent.text
                              if ent.label_ == "CIN NO":
                                cin_no = ent.text
                              if ent.label_ == "COMPANY PAN NO":
                                company_pan = ent.text
                            receipt_tot_amt = table_df[table_df['Serial No'] == 'Total Value']['Amount'].values[0]
                            all_data.append([document_no,company_name,company_name,document_date,company_gstin,vehicle_no,cin_no,company_pan,receipt_tot_amt,row['Serial No'], row['Material Description'], row['HSN/SAC'], row['Qty'], row['Rate'], row['Per'],row['Amount']])
                        else:
                            all_data.append(['','','','','','','','','', row['Serial No'], row['Material Description'], row['HSN/SAC'], row['Qty'], row['Rate'], row['Per'], row['Amount']  ])
                    
                print("_________________________")
                                             

RedBullIndiaPvt.Ltd._4203401401.pdf
4203401401
COMPANY NAME
COMPANY NAME
DOCUMENT DATE
COMPANY GSTIN NUMBER
VEHICLE NO
CIN NO
COMPANY PAN NO
_________________________
SAgarwalEnterprises_4203401412.pdf
4203401412
COMPANY NAME
COMPANY NAME
DOCUMENT DATE
VEHICLE NO
CIN NO
COMPANY PAN NO
_________________________
RedbullIndiaPvtLtdLucknow_4203401414.pdf
4203401414
COMPANY NAME
COMPANY NAME
DOCUMENT DATE
VEHICLE NO
CIN NO
COMPANY PAN NO
_________________________
RedbullIndiaPvtLtdLucknow_4203401410.pdf
4203401410
COMPANY NAME
COMPANY NAME
VEHICLE NO
CIN NO
COMPANY PAN NO
_________________________


In [ ]:
out_df = pd.DataFrame(all_data, columns= ['Document No','COMPANY NAME','COMPANY NAME','DOCUMENT DATE', 'COMPANY GSTIN NUMBER','VEHICLE NO','CIN NO','COMPANY PAN NO', 'Invoice Total Amt','Serial No', 'Material Description', 'HSN/SAC', 'Qty', 'Rate', 'Per','Amount'])

In [ ]:
out_df

,Document No,COMPANY NAME,COMPANY NAME,DOCUMENT DATE,COMPANY GSTIN NUMBER,VEHICLE NO,CIN NO,COMPANY PAN NO,Invoice Total Amt,Serial No,Material Description,HSN/SAC,Qty,Rate,Per,Amount
0,4203401401,RedBull India Pvt. Ltd,RedBull India Pvt. Ltd,04-09-2021,GSTIN:09AADCR5842H1Z6,UP32KN3066,U51909MH2007FTC169580,AADCR5842H,"104,947.03",10,Red Bull Energy Drink IN 250ml\nRB30925\nMfg. ...,22029990,50,734.92,BOX,"36,746.16"
1,,,,,,,,,,20,Red Bull IN Sugar Free 250ml WA\nRB205906\nMfg...,22029990,20,734.92,BOX,"14,698.46"
2,,,,,,,,,,30,Red Bull ED IN Alu Can 24x330ml WA\nRB217737\n...,22029990,40,970.10,BOX,"38,803.95"
3,,,,,,,,,,40,RB Yellow Edition IN Alu Can 24x250ml\nRB23147...,22029990,20,734.92,BOX,"14,698.46"
4,4203401412,RedBull India Pvt. Ltd,RedBull India Pvt. Ltd,21-09-2021,GSTIN:09AADCR5842H1Z6,UP32NN5334,U51909MH2007FTC169580,AADCR5842H,"3,343.30",10,Canc. V2 Wrap BX Purple Ed. Int.\nVD205342030,39199090,2,"1,671.65",NOS,"3,343.30"
5,4203401414,RedBull India Pvt. Ltd,RedBull India Pvt. Ltd,21-09-2021,GSTIN:09AADCR5842H1Z6,UP32NN5334,U51909MH2007FTC169580,AADCR5842H,"14,698.47",10,Red Bull ED IN Alu Can 24x250ml WA\nRB226600\n...,22029990,6,734.92,BOX,"4,409.54"
6,,,,,,,,,,20,Red Bull IN Sugar Free 250ml WA\nRB205906\nMfg...,22029990,2,734.92,BOX,"1,469.85"
7,,,,,,,,,,30,RB Yellow Edition IN Alu Can 24x250ml\nRB23147...,22029990,4,734.92,BOX,"2,939.69"
8,,,,,,,,,,40,RB Purple Edition IN Alu Can 24x250ml\nRB23583...,22029990,8,734.92,BOX,"5,879.39"
9,4203401410,RedBull India Pvt. Ltd,RedBull India Pvt. Ltd,21-09-2021,GSTIN:09AADCR5842H1Z6,UP32LN9531,U51909MH2007FTC169580,AADCR5842H,"2,939.69",10,Red Bull ED IN Alu Can 24x250ml WA\nRB226600\n...,22029990,4,734.92,BOX,"2,939.69"


In [ ]:
out_df.to_csv(OUTPUT_FOLDER+'test_redbull1232.csv')